In [1]:
import xmltodict
from collections import Counter
import copy
import re
import reconstructor

%reload_ext autoreload
%autoreload 2

In [2]:

log_file = "Gie_Bogaert_20130823_17_GA.xml"

text_begin_file = "17_b_Wordlog_original.txt"

text_begin_string = reconstructor.get_begin_text_string(text_begin_file)

print("Length of begin string should be 4501")
print("Length of begin string:", len(text_begin_string))

event_list = reconstructor.get_log_events(log_file)
print("Number of events:", len(event_list))


Length of begin string should be 4501
Length of begin string: 4501
Number of events: 7877


In [3]:

event_type_freq = Counter([event["type"] for event in event_list])
print("Event types\tfrequency")
for event_type, freq in event_type_freq.most_common():
    print(event_type, "\t", freq)

# iterate over events and capture all special keyboard (non-text) outputs
print("\n\n")
print("Special characters:")
for event_window in reconstructor.slide_event_window(event_list):
    if event_window["curr_event"]["type"] != "keyboard":
        continue
    if event_window["curr_event"]["output"] in reconstructor.special_keyboard_outputs:
        #print("skipping:", event["id"], event["type"], event["output"])
        continue
    if len(event_window["curr_event"]["output"]) > 1:
        print(event_window["curr_event"]["id"], event_window["curr_event"]["type"], event_window["curr_event"]["output"])


Event types	frequency
keyboard 	 5616
replacement 	 1367
mouse 	 791
focus 	 101
insert 	 2



Special characters:
108 keyboard LEFT + z
20174 keyboard OEM_6


In [73]:
current_text_string = reconstructor.get_begin_text_string(text_begin_file)
event_list_orig = reconstructor.get_log_events(log_file)
context_size=20

print(len(current_text_string))

4501


In [91]:
current_text_string = reconstructor.get_begin_text_string(text_begin_file)
event_list = copy.deepcopy(event_list_orig)

for event_window in reconstructor.slide_event_window(event_list):
    curr_event = event_window["curr_event"]
    if reconstructor.is_text_load_event(event_window):
        print(event_window["curr_event"]["id"], "\ttext loaded")
        continue
    if reconstructor.text_changes(event_window):
        #print(event_window["curr_event"])
        next_text_string = reconstructor.update_current_text_string(event_window, current_text_string, context_size=context_size)
        #print(next_text_string)
        cursor_position = event_window["curr_event"]["positionFull"]
        next_cursor_context = next_text_string[cursor_position-context_size:cursor_position+context_size+1]
        if curr_event["id"] > 4310:
            print("actual text length:", len(next_text_string))
            print(event_window["curr_event"]["id"], "\ttext update\tcursor position:", event_window["curr_event"]["positionFull"])
            print("------cursor context begin------")
            print(next_cursor_context, "\n")
            print("------cursor context end------\n")
        current_text_string = next_text_string
    else:
        if curr_event["id"] > 4310:
            print(event_window["curr_event"]["id"], "\tno text update\tcursor position:", event_window["curr_event"]["positionFull"])
            print("actual text length:", len(current_text_string))
        pass
    if curr_event["id"] >= 4500:
        break




21 	text loaded
114 TEXT LENGTH DISCREPANCY	expected text length: 4558 	actual text length: 4557
	Inserting newline
682
683
684
685
686
822
1465
1466 	assume propagating correction for delayed update
1468 	assume propagating correction for delayed update
1469 	assume propagating correction for delayed update
2178
3363
3364 	assume propagating correction for delayed update
3366 	assume propagating correction for delayed update
3367 	assume propagating correction for delayed update
3368 	assume propagating correction for delayed update
3369 	assume propagating correction for delayed update
3370 	assume propagating correction for delayed update
3372 CUTTING SELECTED TEXT:  Er is me een nieuw pak beloofd, om me appetijtelijker te maken. (Hoewel beloftes aan een oude man zelden worden ingelost. Ik heb geleerd daarmee te leven.)
3379
3380 	assume propagating correction for delayed update
3382 	assume propagating correction for delayed update
3383 	assume propagating correction for delayed up

In [85]:
re.sub(r"^\[(.*)\]$", r"\1", "[dit klopt [daarom]]")

'dit klopt [daarom]'